In [30]:
import sys
import csv
from itertools import chain, combinations
import math
from collections import defaultdict, Counter
from operator import itemgetter

#test Decision Tree confusion matrices, done and optimized.
bstedt = [[0, 14, 8], [3, 79, 20], [19, 16, 66]]
ntedt = [[1531, 44, 21, 0, 0], [26, 101, 0, 0, 3], [30, 0, 1506, 0, 0], [0, 0, 0, 1605, 0], [0, 0, 0, 0, 0]]
ledtedt = [[272, 79], [81, 702]]
sstedt = [[128, 28, 51, 61], [35, 99, 57, 54], [45, 44, 121, 22], [54, 48, 23, 130]]

#train Decision Tree confusion matrices, done and optimized.
bstrdt = [[27, 0, 0], [0, 186, 0], [0, 0, 187]]
ntrdt = [[2670, 0, 0, 0, 0], [0, 198, 0, 0, 0], [0, 0, 2508, 0, 0], [0, 0, 0, 2715, 0], [0, 0, 0, 0, 2]]
ledtrdt = [[491, 147], [146, 1303]]
sstrdt = [[726, 1, 1, 4], [0, 753, 1, 1], [2, 1, 764, 1], [2, 2, 1, 740]]

#test Random Forest confusion matrices, done and optimized.
bsterf = [[0, 12, 10], [0, 86, 16], [0, 13, 88]]
nterf = [[1546, 6, 44, 0, 0], [67, 63, 0, 0, 0], [35, 0, 1501, 0, 0], [0, 0, 0, 1605, 0], [0, 0, 0, 0, 0]]
ledterf = [[269, 82], [73, 710]]
ssterf = [[182, 5, 37, 44], [8, 174, 25, 38], [23, 22, 183, 4], [25, 24, 3, 203]]

#train Random Forest confusion matrices, done and optimized.
bstrrf = [[27, 0, 0], [0, 186, 0], [0, 0, 187]]
ntrrf = [[2669, 0, 1, 0, 0], [0, 198, 0, 0, 0], [0, 0, 2508, 0, 0], [0, 0, 0, 2715, 0], [0, 0, 0, 0, 2]]
ledtrrf = [[491, 147], [146, 1303]]
sstrrf = [[732, 0, 0, 0], [0, 755, 0, 0], [0, 0, 768, 0], [0, 0, 0, 745]]

In [31]:
def overall_accuracy(matrix):
    print('overall accuracy of set: ')
    total_population = sum(sum(matrix, []))
    diagonals = 0
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                diagonals += matrix[i][j]
    result = float(diagonals)/float(total_population)
    print("%.4f" % result)

In [32]:
def accuracy(matrix):
    print('accuracy metric per label: ')
    total_population = sum(sum(matrix, []))
    for i in range(len(matrix)):
        tp_and_tn = 0
        true_negative = 0
        for j in range(len(matrix)):
            if j == i:
                tp_and_tn += matrix[j][i]
                continue
            true_negative += sum(matrix[j])
            true_negative -= matrix[j][i]
        tp_and_tn += true_negative
        if total_population == 0:
            print('& ', 0.0)
            continue
        #print('tp and tn: ', tp_and_tn, 'total population: ', total_population)
        result = float(tp_and_tn)/float(total_population)
        print('& ', "%.4f" % result, ' ', end='')

In [33]:
def specificity(matrix):
    print('specific metric per label: ')
    for i in range(len(matrix)):
        true_negative = 0
        tn_and_fp = 0
        for j in range(len(matrix)):
            if j == i:
                continue
            tn_and_fp += sum(matrix[j])
            true_negative += sum(matrix[j])
            true_negative -= matrix[j][i]
        #print('true negative: ', true_negative, 'tn and fp: ', tn_and_fp)
        if tn_and_fp == 0:
            print('& ', 0.0)
            continue
        result = (float(true_negative)/float(tn_and_fp))
        print('& ', "%.4f" % result, ' ', end='')

In [34]:
def recall(matrix):
    print('sensitivity metric per label: ')
    for i in range(len(matrix)):
        if float(sum(matrix[i])) == 0:
            print('& ', 0.0)
            continue
        result = (float(matrix[i][i]) / float(sum(matrix[i])))
        print('& ', "%.4f" % result, ' ', end='')

In [35]:
def precision(matrix): 
    print('precision metric per label: ')
    for i in range(len(matrix)):
        true_positive = 0
        tp_and_fp = 0
        for j in range(len(matrix)):
            if i == j:
                true_positive += matrix[i][j]
            tp_and_fp += matrix[j][i]
        if tp_and_fp == 0:
            print('& ', 0.0)
            continue
        result = (float(true_positive)/float(tp_and_fp))
        print('& ', "%.4f" % result, ' ', end='')

In [36]:
def f1_score(matrix):
    print('f1 scores: ')
    for i in range(len(matrix)):
        if float(sum(matrix[i])) == 0:
            print('& ', 0.0)
            continue
        sens = (float(matrix[i][i]) / float(sum(matrix[i])))
        
        true_positive = 0
        tp_and_fp = 0
        for j in range(len(matrix)):
            if i == j:
                true_positive += matrix[i][j]
            tp_and_fp += matrix[j][i]
        if tp_and_fp == 0:
            print('& ', 0.0)
            continue
        prec = (float(true_positive)/float(tp_and_fp))
        if prec+sens == 0:
            print('& ', 0.0)
            continue
        f1 = (2*prec*sens)/(prec+sens)
        print('& ', "%.4f" % f1, ' ', end='')

In [37]:
def fb(matrix, beta):
    print('fb scores: ', beta)
    for i in range(len(matrix)):
        if float(sum(matrix[i])) == 0:
            print('& ', 0.0)
            continue
        sens = (float(matrix[i][i]) / float(sum(matrix[i])))
        
        true_positive = 0
        tp_and_fp = 0
        for j in range(len(matrix)):
            if i == j:
                true_positive += matrix[i][j]
            tp_and_fp += matrix[j][i]
        if tp_and_fp == 0:
            print('& ', 0.0)
            continue
        prec = (float(true_positive)/float(tp_and_fp))
        if prec+sens == 0:
            print('& ', 0.0)
            continue
        result = ((1+(beta**2))*prec*sens)/((beta**2)*prec+sens)
        print('& ', "%.4f" % result, ' ', end='')

In [57]:
#need to test all metrics for both training and test data sets.
#data = extract_data_from_file('balance.scale.train')
#test_data = extract_data_from_file('balance.scale.test')
#attribute_list = create_attribute_list(data)
#root = generate_decision_tree(data, attribute_list)
#predicted_labels = predict_labels(test_data, root)
#matrix = create_confusion(test_data, predicted_labels)
matrix = sstrrf
overall_accuracy(matrix)
accuracy(matrix)
recall(matrix)
precision(matrix)
specificity(matrix)
f1_score(matrix)
fb(matrix, 0.5)
fb(matrix, 2)

overall accuracy of set: 
1.0000
accuracy metric per label: 
&  1.0000  &  1.0000  &  1.0000  &  1.0000  sensitivity metric per label: 
&  1.0000  &  1.0000  &  1.0000  &  1.0000  precision metric per label: 
&  1.0000  &  1.0000  &  1.0000  &  1.0000  specific metric per label: 
&  1.0000  &  1.0000  &  1.0000  &  1.0000  f1 scores: 
&  1.0000  &  1.0000  &  1.0000  &  1.0000  fb scores:  0.5
&  1.0000  &  1.0000  &  1.0000  &  1.0000  fb scores:  2
&  1.0000  &  1.0000  &  1.0000  &  1.0000  